In [1]:
import numpy as np
import pandas as pd

# ***********Start Data aggregation
# Data is is currently in the form of many samples of the same models at different periods of time
# Aggregating in datal lake could of been done but with lack of user rights to data lake it is easier to do it here.
#
# *******************

In [2]:
# ***********START general data clean up and validation  is done in mqi_data_cleanup.ipynb*******************
model_actions = pd.read_pickle("mqi_data_clean/model_actions_clean_cpy.pkl")
model_cases = pd.read_pickle("mqi_data_clean/model_cases_clean.pkl")
model_states = pd.read_pickle("mqi_data_clean/model_state_clean_cpy.pkl")
model_info = pd.read_pickle("mqi_data_clean/model_info_clean_cpy.pkl")


In [3]:
# ***********START MODEL INFO *******************


def model_info_aggregate():
    # MODEL AGE
    modelinfo_age = pd.DataFrame()
    modelinfo_age["age_inweeks"] = model_info.groupby("modelid", group_keys=False)[
        ["modelid"]
    ].count()

    # GET MODEL CURRENT ACTIVE STATUS:
    modelinfo_active = pd.DataFrame()
    modelinfo_active[
        [
            "currently_active",
            "modeltype",
            "position_code",
            "position_description",
            "site",
            "position_class",
        ]
    ] = model_info.groupby("modelid", group_keys=False)[
        [
            "current_active_value",
            "modeltype",
            "position_code",
            "position_description",
            "site",
            "position_class",
        ]
    ].first()

    modelinfo_active = pd.get_dummies(modelinfo_active, columns=["currently_active"])

    # MODEL SAVE COUNT: calc frequency with model_state table age, see below for more detail
    modelinfo_savecount = pd.DataFrame()
    modelinfo_savecount["model_save_count"] = model_info.groupby("modelid")[
        "lastsavetime"
    ].nunique()

    # MODEL RETRAIN COUNT: calc frequency with model_state table age, see below for more detail
    modelinfo_rebuildcount = pd.DataFrame()
    modelinfo_rebuildcount["model_build_count"] = model_info.groupby("modelid")[
        "lastbuildtime"
    ].nunique()

    # MODEL SCORE
    modelinfo_score = model_info.groupby("modelid", group_keys=False)[
        [
            "fixedlimitscore",
            "gffscore",
            "linregscore",
            "logregscore",
            "rollingaveragescore",
        ]
    ].mean()
    modelinfo_score["model_score_avg"] = modelinfo_score.mean(axis=1)

    # combine all aggregations
    final_info = pd.concat(
        [
            modelinfo_age,
            modelinfo_active,
            modelinfo_savecount[["model_save_count"]],
            modelinfo_rebuildcount[["model_build_count"]],
            modelinfo_score[["model_score_avg"]],
        ],
        axis=1,
    )

    # add alternative grouping for equipment type
    final_info["pos_class_short"] = final_info["position_class"].str.split(
        n=1, pat="-", expand=True
    )[0]
    final_info["pos_class_short"] = final_info["pos_class_short"].str.strip()

    unique_groups = (
        final_info.groupby("pos_class_short")["position_class"].unique().apply(list)
    )

    unique_groups["MP-PUMP"] = ["MP-PUMP"]
    unique_groups["MP-COMP"] = ["MP-COMP"]

    remlst = ["MP-PUMP", "MP-COMP"]
    [unique_groups["MP"].remove(item) for item in remlst]
    unique_groups.rename(index={"MP": "MP-FAN_TURBN_GRBX_OTHER"}, inplace=True)

    unique_groups["FP-FIRED"] = ["FP-FIRED"]
    unique_groups["FP-EXCH"] = ["FP-EXCH"]
    unique_groups["FP-tank-vessel"] = ["FP-TANK", "FP-VESSL"]

    remlst = ["FP-FIRED", "FP-EXCH", "FP-TANK", "FP-VESSL"]
    [unique_groups["FP"].remove(item) for item in remlst]
    unique_groups.rename(index={"FP": "FP-COMP_VALVE_PIPE_OTHER"}, inplace=True)

    unique_groups["IP-VALVE"] = ["IP-VALVE"]
    unique_groups["IP"].remove("IP-VALVE")
    unique_groups.rename(index={"IP": "IP-LOOPS_XMTR_OTHER"}, inplace=True)

    unique_groups["OP"] = unique_groups["OP"] + unique_groups["P"]
    unique_groups.drop("P", inplace=True)
    unique_groups.rename(index={"OP": "OP_P"}, inplace=True)

    for newcls, clslst in zip(unique_groups.index, unique_groups):
        final_info.loc[
            final_info["position_class"].isin(clslst), "position_class_group"
        ] = newcls

    # drop unused columns
    final_info.drop(
        columns=[
            "position_class",
            "pos_class_short",
            "position_description",
            "position_code",
        ],
        inplace=True,
    )

    final_info.loc[final_info['position_class_group'].isna(), 'position_class_group'] = 'unknown'
    final_info.loc[final_info['site'].isna(), 'site'] = 'unknown'
    return final_info


final_info = model_info_aggregate()
# ***********END MODEL INFO*******************


In [4]:
# ***********START MODEL ACTIONS *******************
def model_actions_aggregate():
    # get averages of model values for 'actual', 'expected', 'upper', and 'lower':
    # this is not used in final table instead using  model_state table, not adding because there are not values for every model ID like model state
    model_values = model_actions[
        ["modelid", "actual", "expected", "upper", "lower"]
    ].copy()
    model_values = model_values.groupby("modelid")[
        ["actual", "expected", "upper", "lower"]
    ].mean()

    # TOUCH COUNT: add number of touches, ie. how many total times any action was performed to a model. NOTE Every row is an action performed
    model_touches = model_actions.copy()
    model_touches = model_actions.groupby("modelid", group_keys=False)[
        ["modelid"]
    ].count()
    model_touches = model_touches.rename(columns={"modelid": "touches_count"})

    # LENGTH OF NOTE: get average length of notes for each action on each model ID
    mod_comnt_temp = model_actions[["modelid", "actionnote"]].copy()
    mod_comnt_temp["actionnoteLen"] = model_actions["actionnote"].apply(len)
    model_comments_len = mod_comnt_temp.groupby("modelid", group_keys=False)[
        ["actionnoteLen"]
    ].mean()

    model_comments_len.rename(
        columns={"actionnoteLen": "action_note_len_avg"}, inplace=True
    )

    # ACTION TYPE COUNT: get count of each type of actiontype for each Model ID.
    actionTypes_df = model_actions[["modelid", "actiontype"]].copy()
    actionTypes_df = actionTypes_df.set_index("modelid")
    # create columns for all values
    actionTypes_df = pd.get_dummies(actionTypes_df, columns=["actiontype"])
    actionTypes_df = actionTypes_df.groupby("modelid", group_keys=False).sum()

    #combine dataframes
    return pd.concat(
        [model_values, model_comments_len, actionTypes_df, model_touches], axis=1
    )


final_actions_df = model_actions_aggregate()

# ***********END MODEL ACTIONS*******************


In [5]:
# ***********START MODEL STATES *******************
def model_state_aggregate():
    # AGG ALERT TYPES: alerts are doubled up, ie on cell may have 'AAS,HHA' and another may have 'HHA' so this creates many columns
    alertTypes_df = model_states[["modelid", "activealerts"]].copy()
    alertTypes_df = alertTypes_df.set_index("modelid")

    unique_alerts = alertTypes_df["activealerts"].unique()
    unique_alerts_lst = list(
        set([alert for alerts in unique_alerts for alert in alerts.split(",")])
    )
    alertTypes_df[unique_alerts_lst] = 0
    for i in range(len(unique_alerts_lst)):
        alertTypes_df.loc[
            alertTypes_df["activealerts"].str.contains(unique_alerts_lst[i]),
            unique_alerts_lst[i],
        ] = 1
    # sum up all each alert type for each model ID
    alertTypes_df = alertTypes_df.groupby("modelid", group_keys=False).sum()

    #Calculate model age. Each row is a week (samples taken at 1 week interval)
    alertTypes_df["model_age"] = (
        model_states.groupby("modelid", group_keys=False)["modelid"].count().copy()
    )

    alertTypes_df['in_alert'] = alertTypes_df[['AAF', 'LLA', 'OSC', 'WOR', 'AAS', 'HHA', 'FRQ']].sum(axis=1)
    #NOTE: "no alerts" was set for all null values in "active alerts column" in mqi_data_clean


    #!when looking at data noticed that actual_vs_expected_percdiff, upper_vs_expected_percdiff and lower_vs_expected_percdiff had alot of values == 200
    #this is when actual and expected are 0 (or very near 0), possible remove these values before doing calc, for leave in and see if maybe it is a quality signifier in cluster
    #! fixed limits will have same value for actual and expected
    # AGG AVG VALUES: convert values to percent difference more important is how model is predicting relative to self(ie. actual vs expected)
    values_lst = ["actual", "expected", "upper", "lower"]
    state_values = model_states.groupby("modelid")[values_lst].mean().copy()
    state_values_orig = state_values.copy()

    state_values["actual_vs_expected_percdiff"] = (
        ((state_values["actual"] - state_values["expected"]).abs())
        / (((state_values["actual"] + state_values["expected"]).abs())/ 2)
    ) * 100

    state_values["upper_vs_actual_percdiff"] = (
        ((state_values["upper"] - state_values["actual"]).abs())
        / (((state_values["upper"] + state_values["actual"]).abs()) / 2)
    ) * 100
    state_values["lower_vs_actual_percdiff"] = (
        ((state_values["lower"] - state_values["actual"]).abs())
        / (((state_values["lower"] + state_values["actual"]).abs()) / 2)
    ) * 100

    # remove old columns and fill null values with 0 (if avg values are 0 percdiff will make it null, all rows before grouping had NO NULL values)
    state_values.drop(["actual", "expected", "upper", "lower"], axis=1, inplace=True)
    # state_values.fillna(0, inplace=True)

    return pd.concat([alertTypes_df, state_values], axis=1)


final_state_df = model_state_aggregate()
# ***********END MODEL STATES *******************


C:\Users\jordom2\AppData\Local\Temp\ipykernel_6872\3235251916.py:18: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  alertTypes_df = alertTypes_df.groupby("modelid", group_keys=False).sum()


In [6]:
# ***********START CASES STATES *******************
def model_cases_aggregate(model_cases, model_casecount):

    # group by case ID to get count of each equipment case
    model_casesgroup = model_cases.groupby("casemgmt_equipment_id", group_keys=False)[
        ["casemgmt_equipment_id"]
    ].count()

    # map equipment ID back to final_info DF to get case count
    model_casecount["case_count"] = model_casecount["position_code"].map(
        model_casesgroup["casemgmt_equipment_id"]
    )

    print(
        f"Found: {model_casecount['case_count'].sum()} cases of a total {model_casesgroup['casemgmt_equipment_id'].sum()} cases"
    )

    model_casecount.drop(columns='position_code', inplace=True)



    return model_casecount


#!SHOULD BE ABLE TO REPLACE WITH FINAL INFO
# add these columns after model_info is aggregated, use to link cases and equipment type
model_info_addedcols = pd.read_pickle(
    "mqi_data_clean/model_info_clean_added_col_3.pkl"
)
final_case_df = model_cases_aggregate(model_cases, model_info_addedcols[['position_code']].copy())


Found: 14884.0 cases of a total 2786 cases


In [7]:
# ***********Start ASSEMBLE Aggregated DF *******************
def merge_all_tables():
    # combine model_info and model_state aggregated tables
    aggreg_df = pd.merge(
        final_info, final_state_df, left_index=True, right_index=True, how="left"
    )

    aggreg_df = pd.merge(
        aggreg_df, final_case_df, left_index=True, right_index=True, how="left"
    )

    # get names for columns from actions table to add to aggregated_df, combine model_actions to aggregated table
    actions_df_col = list(final_actions_df.columns)
    actions_df_col = actions_df_col[4 : len(actions_df_col)]
    aggreg_df = pd.merge(
        aggreg_df,
        final_actions_df[actions_df_col],
        left_index=True,
        right_index=True,
        how="left",
    )

    return aggreg_df


aggreg_df = merge_all_tables()
aggreg_df.shape


(22101, 39)

In [8]:
"""
Difference looks due to model_info table missing random rows compared to model_state. 
Difference is small, worst case is 3% different (only 3 cases near this percent).
    ACTION: use model_state "model_age", this value will better align with more aggregated rows (ie. 'actual, expected, etc.)
"""
def check_mod_age():
    model_age_check = aggreg_df[["age_inweeks", "model_age"]].copy()
    model_age_check["combined"] = aggreg_df["age_inweeks"] == aggreg_df["model_age"]
    model_age_check["how_off"] = aggreg_df["age_inweeks"] - aggreg_df["model_age"]

    print(f"average amount age off:{model_age_check['how_off'].mean()}")
    print(f"max amount age off:{model_age_check['how_off'].max()}")
    print(f"number of values off: {model_age_check[['combined']].value_counts()}")

check_mod_age()

aggreg_df.drop(columns=["age_inweeks"], inplace=True)


average amount age off:0.00936609203203475
max amount age off:4
number of values off: combined
True        18782
False        3319
dtype: int64


In [9]:
aggreg_df.columns

Index(['modeltype', 'site', 'currently_active_False', 'currently_active_True',
       'model_save_count', 'model_build_count', 'model_score_avg',
       'position_class_group', 'LLA', 'AAS', 'AAF', 'noAlert', 'WOR', 'FRQ',
       'HHA', 'OSC', 'model_age', 'in_alert', 'actual_vs_expected_percdiff',
       'upper_vs_actual_percdiff', 'lower_vs_actual_percdiff', 'case_count',
       'action_note_len_avg', 'actiontype_Clear Alert Status',
       'actiontype_Diagnose Cleared', 'actiontype_Diagnose Set',
       'actiontype_Ignore Expiration', 'actiontype_Ignore Set',
       'actiontype_Model Maintenance Cleared',
       'actiontype_Model Maintenance Set', 'actiontype_Note Added',
       'actiontype_Quick Watch Set', 'actiontype_Stop Ignoring',
       'actiontype_Watch Cleared', 'actiontype_Watch Expiration',
       'actiontype_Watch Override', 'actiontype_Watch Set', 'touches_count'],
      dtype='object')

In [10]:
#convert to frequency: Models created in past could collect alerts so use freq, each row ingested 1 time a week, count of model ID == model age
def model_alerts_to_freq():
    col_lst = [
        'WOR', 'HHA', 'OSC', 'FRQ', 'noAlert', 'LLA',
       'AAS', 'AAF', 'in_alert'
        ]
    col_lst_new = [(str(col) + "_freq") for col in col_lst]

    # convert To action counts to Frequency using model_age from model_state aggregations
    aggreg_df[col_lst] = aggreg_df[col_lst].div(aggreg_df["model_age"].values, axis=0)
    aggreg_df.rename(columns=dict(zip(col_lst, col_lst_new)), inplace=True)

    # if actions is 0 .div ends up in with null value, replace with 0
    aggreg_df[col_lst_new] = aggreg_df[col_lst_new].fillna(0)



def model_actions_to_freq():
    col_lst = [
        'actiontype_Clear Alert Status',
        'actiontype_Diagnose Cleared',
        'actiontype_Diagnose Set',
        'actiontype_Ignore Expiration',
        'actiontype_Ignore Set',
        'actiontype_Model Maintenance Cleared',
        'actiontype_Model Maintenance Set',
        'actiontype_Note Added',
        'actiontype_Quick Watch Set',
        'actiontype_Stop Ignoring',
        'actiontype_Watch Cleared',
        'actiontype_Watch Expiration',
        'actiontype_Watch Override',
        'actiontype_Watch Set',
        'touches_count'
        ]
    # col_lst = list(final_actions_df.columns)
    # col_lst = col_lst[5 : len(col_lst)]
    col_lst_new = [(str(col) + "_freq") for col in col_lst]

    # convert To action counts to Frequency using model_age from model_state aggregations
    aggreg_df[col_lst] = aggreg_df[col_lst].div(aggreg_df["model_age"].values, axis=0)
    aggreg_df.rename(columns=dict(zip(col_lst, col_lst_new)), inplace=True)

    # if actions is 0 .div ends up in with null value, replace with 0
    aggreg_df[col_lst_new] = aggreg_df[col_lst_new].fillna(0)


def model_info_to_freq():
    col_lst = ["model_save_count", "model_build_count"]
    col_lst_new = ["model_save_freq", "model_build_freq"]
    aggreg_df[col_lst] = aggreg_df[col_lst].div(aggreg_df["model_age"].values, axis=0)
    aggreg_df.rename(columns=dict(zip(col_lst, col_lst_new)), inplace=True)

def model_cases_to_freq():
    col_lst = ["case_count"]
    col_lst_new = ["case_count_freq"]

    #add new column to keep case count for final metrics 
    aggreg_df[col_lst_new] = aggreg_df[col_lst].div(aggreg_df["model_age"].values, axis=0)
    # aggreg_df.rename(columns=dict(zip(col_lst, col_lst_new)), inplace=True)
    # fill any null values for cases
    aggreg_df[["case_count_freq"]] = aggreg_df[["case_count_freq"]].fillna(0)


model_alerts_to_freq()
model_actions_to_freq()
model_info_to_freq()
model_cases_to_freq()

#some models had no action note comments, so after merge value was null
aggreg_df['action_note_len_avg'] = aggreg_df['action_note_len_avg'].fillna(0)
aggreg_df['case_count'] = aggreg_df['case_count'].fillna(0)

In [11]:
aggreg_df.columns

Index(['modeltype', 'site', 'currently_active_False', 'currently_active_True',
       'model_save_freq', 'model_build_freq', 'model_score_avg',
       'position_class_group', 'LLA_freq', 'AAS_freq', 'AAF_freq',
       'noAlert_freq', 'WOR_freq', 'FRQ_freq', 'HHA_freq', 'OSC_freq',
       'model_age', 'in_alert_freq', 'actual_vs_expected_percdiff',
       'upper_vs_actual_percdiff', 'lower_vs_actual_percdiff', 'case_count',
       'action_note_len_avg', 'actiontype_Clear Alert Status_freq',
       'actiontype_Diagnose Cleared_freq', 'actiontype_Diagnose Set_freq',
       'actiontype_Ignore Expiration_freq', 'actiontype_Ignore Set_freq',
       'actiontype_Model Maintenance Cleared_freq',
       'actiontype_Model Maintenance Set_freq', 'actiontype_Note Added_freq',
       'actiontype_Quick Watch Set_freq', 'actiontype_Stop Ignoring_freq',
       'actiontype_Watch Cleared_freq', 'actiontype_Watch Expiration_freq',
       'actiontype_Watch Override_freq', 'actiontype_Watch Set_freq',
     

In [12]:
final_info.to_pickle("mqi_data_aggregated/final_info_df.pkl")
final_state_df.to_pickle("mqi_data_aggregated/final_state_df.pkl")
final_actions_df.to_pickle("mqi_data_aggregated/final_actions_df.pkl")
aggreg_df.to_pickle("mqi_data_aggregated/aggreg_df.pkl")
